# Square vs Elu, Mean vs Max Pool

I have tested following variants for our shallownet:

* square or elu nonlinearity
* mean or max pooling

After squaring, the model takes the logarithm of the pooled output, i.e. square+log in tables below either means square+max+log or square+mean+log.

I tested for:
* 0-4 Hz: 4 Hz lowpass
* $>$0 Hz: No low/high/bandpass at all
* $>$4 Hz: 4 Hz highpass


Bolded are the results from our current model (i.e. square+mean+log).

Be aware:
* Samplewise trained models have an input window of ~2 seconds to predict a sample
* Trialwise trained models use the entire 4 second trial to make a single prediction for the trial


Overall results are still open to interpretation I think:
* square+mean+log better for higher frequencies, elu+max better for lower frequencies (exception: bci competition trialwise for higher frequencies, somewhat bci competition samplewise for lower frequencies)

One consistent description would be:

* square+mean+log has higher accuracies on high frequencies trained samplewise
* elu+max has higher accuracies using the full trial on the low frequencies

=> Elu+max better at using potentials that typically occur at start of trial? Square+mean+log better at using oscillations?

## BCI Competition

### Trialwise Training

|band|pool|square+log|elu|
|-|
|0-4|mean|**58.9**|60.6|
|0-4|max|56.9|64.6|
|>0|mean|**69.8**|63.9|
|>0|max|65.8|73.0|
|>4|mean|**68.5**|73.0|
|>4|max|65.1|63.4|

### Samplewise Training

|band|pool|square+log|elu|
|-|
|0-4|mean|**61.4**|48.1|
|0-4|max|61.7|59.7|
|>0|mean|**74.4**|64.8|
|>0|max|71.2|67.9|
|>4|mean|**67.8**|66.0|
|>4|max|69.1|64.0|

## Ours

### Trialwise training

|band|pool|square+log|elu|
|-|
|0-4|mean|**76.3**|78.0|
|0-4|max|82.1|83.3|
|>0|mean|**89.3**|83.9|
|>0|max|89.8|90.9|
|>4|mean|**91.4**|90.9|
|>4|max|88.2|87.6|

### Samplewise Training

|band|pool|square+log|elu|
|-|
|0-4|mean|**75.7**|74.5|
|0-4|max|78.3|82.8|
|>0|mean|**87.8**|84.4|
|>0|max|87.4|87.8|
|>4|mean|**93.0**|88.2|
|>4|max|89.9|88.6|

# Appendix: Results including square without log

 I also tested without taking the logarithm but that was mostly either similar or much worse (even training not stable in some cases).

## BCI Competition

### Trialwise Training

|band|pool|square+log|square|elu|
|-|
|0-4|mean|**58.9**|53.2|60.6|
|0-4|max|56.9|50.5|64.6|
|>0|mean|**69.8**|67.9|63.9|
|>0|max|65.8|48.0|73.0|
|>4|mean|**68.5**|65.4|73.0|
|>4|max|65.1|43.4|63.4|

### Samplewise Training

|band|pool|square+log|square|elu|
|-|
|0-4|mean|61.4|58.9|48.1|
|0-4|max|61.7|60.7|59.7|
|>0|mean|74.4|72.7|64.8|
|>0|max|71.2|64.1|67.9|
|>4|mean|67.8|69.8|66.0|
|>4|max|69.1|61.9|64.0|

## Ours

### Trialwise training

|band|pool|square+log|square|elu|
|-|
|0-4|mean|**76.3**|76.8|78.0|
|0-4|max|82.1|78.1|83.3|
|>0|mean|**89.3**|88.7|83.9|
|>0|max|89.8|88.3|90.9|
|>4|mean|91.4|90.2|90.9|
|>4|max|88.2|81.7|87.6|

### Samplewise Training

|band|pool|square+log|square|elu|
|-|
|0-4|mean|**75.7**|77.3|74.5|
|0-4|max|78.3|78.5|82.8|
|>0|mean|**87.8**|88.2|84.4|
|>0|max|87.4|87.6|87.8|
|>4|mean|**93.0**|92.9|88.2|
|>4|max|89.9|88.3|88.6|

In [3]:
%%capture
import os
import site
site.addsitedir('/home/schirrmr/.local/lib/python2.7/site-packages/')
site.addsitedir('/usr/lib/pymodules/python2.7/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
%cd /home/schirrmr/braindecode/code/braindecode/
assert 'THEANO_FLAGS' in os.environ
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'
%load_ext autoreload
%autoreload 2
import numpy as np

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png' 

import matplotlib.lines as mlines
import seaborn
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
matplotlib.rcParams['font.size'] = 7
seaborn.set_style('darkgrid')

from braindecode.scripts.print_results import ResultPrinter
from braindecode.csp.print_results import CSPResultPrinter
import logging
log = logging.getLogger()
log.setLevel("DEBUG")